<a href="https://colab.research.google.com/github/chrispoole70/our-daily-stretch/blob/main/our_daily_stretch_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal for this agent is to be a stretching coach. The user could describe what's causing them discomfort and ask the coach for stretches to do. Then the coach could share a daily stretching plan with them and send a reminder to stretch.

---

**Here's an example ideal conversation:**
* Human: Hello, I'm feeling discomfort in my hips. I'm hoping you can help me create a stretching routine to alleviate some of this pain and improve my flexibility.
* AI: Hello, of course I can help you with that! I know a lot of good hip stretches that you can try. First, I'd like to know how much time you'd like to allot for stretching. Is five minutes ok?
* Human: Sure, five minutes sounds good.
* AI: Ok. Here are five hip stretches you could do for one minute each:
   1. _Stretch 1_
   2. _Stretch 2_
   3. _Stretch 3_
   4. _Stretch 4_
   5. _Stretch 5_

  How does this stretch plan look?
* Human: I think it looks good, let me give it a try.
* AI: Sure thing! Would you like me to send you a reminder to stretch?
* Human: Yeah, that would be helpful.
* AI: Great, I will send this stretch plan to you as a reminder. Would you like me to send it as a text message to your phone?
* Human: Yeah, that works.
* AI: Great, and how often would you like to stretch? I can send you a reminder everyday, for example.
* Human: Stretching everyday would be a good goal for me. Sure, you can send me a reminder everyday before I go to bed, around 9PM.
* AI: Absolutely! I will send this stretch plan to you tonight around 9PM via text message. Then I will send subsequent reminders everyday at 9PM. Good luck with the stretching and please let me know how it goes! I'm here to help if you want to revise your plan.
* Human: Thanks! I'll do the stretches and check in with you later.

---

**Here's a short conversation to get started with:**
* Human: Can you create a stretching routine for me? I'd like to focus on hip stretches to improve my flexibility. The plan should have five stretches that I can do for one minute each.
* AI: No problem! Here are five hip stretches you can do for one minute each:
   1. _Stretch 1_
   2. _Stretch 2_
   3. _Stretch 3_
   4. _Stretch 4_
   5. _Stretch 5_


In [ ]:
%pip list

Package                               Version
------------------------------------- -------------------
absl-py                               1.4.0
accelerate                            1.5.2
aiohappyeyeballs                      2.6.1
aiohttp                               3.11.15
aiosignal                             1.3.2
alabaster                             1.0.0
albucore                              0.0.23
albumentations                        2.0.5
ale-py                                0.10.2
altair                                5.5.0
annotated-types                       0.7.0
anyio                                 4.9.0
argon2-cffi                           23.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.1
arviz                                 0.21.0
astropy                               7.0.1
astropy-iers-data                     0.2025.4.14.0.37.22
astunparse                            1.6.3
atpublic                              5

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph

In [ ]:
%pip list

Package                               Version
------------------------------------- -------------------
absl-py                               1.4.0
accelerate                            1.5.2
aiohappyeyeballs                      2.6.1
aiohttp                               3.11.15
aiosignal                             1.3.2
alabaster                             1.0.0
albucore                              0.0.23
albumentations                        2.0.5
ale-py                                0.10.2
altair                                5.5.0
annotated-types                       0.7.0
anyio                                 4.9.0
argon2-cffi                           23.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.1
arviz                                 0.21.0
astropy                               7.0.1
astropy-iers-data                     0.2025.4.14.0.37.22
astunparse                            1.6.3
atpublic                              5

In [ ]:
import os, getpass
from google.colab import userdata

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = userdata.get(var)

_set_env("OPENAI_API_KEY")

# Short Conversation

First, let's reverse engineer the application by starting with the short human conversation...

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

messages = [HumanMessage(content="Can you create a stretching routine for me? I'd like to focus on stretches to improve my flexibility. The plan should have five stretches that I can do for one minute each.")]
# messages = react_graph.invoke({"messages": messages})

Next, let's instantiate our LLM...

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

Based on the short human message, we know that there will need to be a tool call to get the list of stretches...

In [ ]:
from typing import List

def get_stretches() -> List[str]:
  ''' Returns a list of stretches
  '''
  return [
      'Archer Pose',
      'Four Corner Stretch',
      'Revolved Paying Homage',
      'Reverse Prayer',
      'Root Lock'
      ]

tools = [get_stretches]

In [ ]:
llm_bound_to_tools = llm.bind_tools(tools=tools)

Then let's write down a system message with instructions for the LLM...

In [ ]:
# System message
system_msg = SystemMessage(content="""
You are a stretch coach tasked with generating stretch plans. If a user asks for
stretches, only use the stretches returned from the tool called get_stretches.
""")

def stretch_coach(state: MessagesState):
  return {
      'messages': [llm_bound_to_tools.invoke(f'{system_msg} {state["messages"]}')]
  }

In order to invoke the message, we need to:
1. Build the graph
2. Add nodes to the graph
3. Connect the nodes via edges

In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

# 1. Build the graph
graph_builder = StateGraph(MessagesState)

# 2. Add nodes to the graph
graph_builder.add_node('stretch_coach', stretch_coach)
graph_builder.add_node('tools', ToolNode(tools))

# 3. Connect the nodes via edges
graph_builder.add_edge(start_key=START, end_key='stretch_coach')
graph_builder.add_conditional_edges(source='stretch_coach', path=tools_condition)
graph_builder.add_edge(start_key='tools', end_key='stretch_coach')

# Lastly, compile the graph so that it can be invoked
runnable_graph = graph_builder.compile()

In [ ]:
# Displaying the graph within the notebook is not working. I tried to generate it locally instead of calling the Mermaid API
# but that hasn't worked yet, either: https://langchain-ai.github.io/langgraph/how-tos/visualization/#using-mermaidink

# from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

# display(Image(runnable_graph.get_graph(xray=True).draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))

Can copy the YAML below and paste it into the [Mermaid live editor](https://mermaid.live/)

In [ ]:
print(runnable_graph.get_graph(xray=True).draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	stretch_coach(stretch_coach)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> stretch_coach;
	tools --> stretch_coach;
	stretch_coach -.-> tools;
	stretch_coach -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
messages = runnable_graph.invoke({'messages': messages})

In [ ]:
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Can you create a stretching routine for me? I'd like to focus on stretches to improve my flexibility. The plan should have five stretches that I can do for one minute each.
================================== Ai Message ==================================
Tool Calls:
  get_stretches (call_A01RbYv0SWV7u8QPnRnZI2nt)
 Call ID: call_A01RbYv0SWV7u8QPnRnZI2nt
  Args:
================================= Tool Message =================================
Name: get_stretches

["Archer Pose", "Four Corner Stretch", "Revolved Paying Homage", "Reverse Prayer", "Root Lock"]
================================== Ai Message ==================================

Here's a stretching routine focusing on improving your flexibility. Perform each stretch for one minute:

1. **Archer Pose**: Stand with your feet wide apart, turn your right foot out, and bend your right knee. Extend your arms parallel to the ground, one forward and the othe

TODO:
* I don't think we want the LLM to generate instructions for each stretch in real-time. I should probably write instructions for each stretch when I record it. Or maybe I could have an LLM watch the video and generate instructions?
* In order for the longer ideal conversation to be able to be had, I need to:
   1. Add memory so that the state of each graph invocation is kept.
   2. Update the `get_stretches` tool to return stretches that are tagged with the body part the user wants to stretch.
   3. Add a tool for scheduling a reminder.
   4. Should the agent be given goals? Such as, goal 1 is figure out what types of stretches, goal 2 is to figure how many stretches, goal 3 is to figure out how long each stretch should be, etc.


# Longer Conversation

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

messages = [HumanMessage(content="Hello, I'm feeling discomfort in my hips. I'm hoping you can help me create a stretching routine to alleviate some of this pain and improve my flexibility.")]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from typing import List

def get_stretches() -> List[str]:
  ''' Returns a list of stretches
  '''
  return [
      'Archer Pose',
      'Four Corner Stretch',
      'Revolved Paying Homage',
      'Reverse Prayer',
      'Root Lock'
      ]

tools = [get_stretches]

In [ ]:
llm_bound_to_tools = llm.bind_tools(tools=tools)

In [ ]:
# System message
system_msg = SystemMessage(content="""
You are a stretch coach tasked with generating stretch plans. Here are your specific instructions:

* If a user asks for stretches, only use the stretches returned from the tool called get_stretches.
* When generating a stretch plan, include the stretch names but not descriptions of how to do the stretches.
* You have these goals to achieve in a conversation:
   1. Ask how long they would like to stretch for. 5 minutes is the default length of time.
   2. Ask how often they would like to stretch for. Daily is the default cadence.
   3. Generate a stretch plan that adheres to the user's time limit and cadence.


Below is an example of a conversation a user can have with you:

Human: Hello, I'm feeling discomfort in my hips. I'm hoping you can help me create a stretching routine to alleviate some of this pain and improve my flexibility.

AI: Hello, of course I can help you with that! I know a lot of good hip stretches that you can try. First, I'd like to know how much time you'd like to allot for stretching. Is five minutes ok?

Human: Sure, five minutes sounds good.

AI: Ok. Here are five hip stretches you could do for one minute each:
   * Stretch 1
   * Stretch 2
   * Stretch 3
   * Stretch 4
   * Stretch 5
How does this stretch plan look?

Human: Thanks, I think it looks good.

AI: Great, and how often would you like to stretch? Once a day?

Human: Stretching everyday would be a good goal for me. Let me try out these stretches later today.

AI: Sounds good, I hope it goes well!
""")

def stretch_coach(state: MessagesState):
  return {
      'messages': [llm_bound_to_tools.invoke(f'{system_msg} {state["messages"]}')]
  }

In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

# 1. Build the graph
graph_builder = StateGraph(MessagesState)

# 2. Add nodes to the graph
graph_builder.add_node('stretch_coach', stretch_coach)
graph_builder.add_node('tools', ToolNode(tools))

# 3. Connect the nodes via edges
graph_builder.add_edge(start_key=START, end_key='stretch_coach')
graph_builder.add_conditional_edges(source='stretch_coach', path=tools_condition)
graph_builder.add_edge(start_key='tools', end_key='stretch_coach')

# Lastly, compile the graph so that it can be invoked
runnable_graph = graph_builder.compile()

In [ ]:
print(runnable_graph.get_graph(xray=True).draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	stretch_coach(stretch_coach)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> stretch_coach;
	stretch_coach -.-> tools;
	tools --> stretch_coach;
	stretch_coach -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
messages

[HumanMessage(content="Hello, I'm feeling discomfort in my hips. I'm hoping you can help me create a stretching routine to alleviate some of this pain and improve my flexibility.", additional_kwargs={}, response_metadata={})]

In [ ]:
messages = runnable_graph.invoke({'messages': messages})

In [ ]:
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Hello, I'm feeling discomfort in my hips. I'm hoping you can help me create a stretching routine to alleviate some of this pain and improve my flexibility.
================================== Ai Message ==================================
Tool Calls:
  get_stretches (call_Fdjj80irrLURazobJzXS2KSQ)
 Call ID: call_Fdjj80irrLURazobJzXS2KSQ
  Args:
================================= Tool Message =================================
Name: get_stretches

["Archer Pose", "Four Corner Stretch", "Revolved Paying Homage", "Reverse Prayer", "Root Lock"]
================================== Ai Message ==================================

Hello, of course I can help you with that! I know a lot of good hip stretches that you can try. First, I'd like to know how much time you'd like to allot for stretching. Is five minutes ok?


In [ ]:
messages

{'messages': [HumanMessage(content="Hello, I'm feeling discomfort in my hips. I'm hoping you can help me create a stretching routine to alleviate some of this pain and improve my flexibility.", additional_kwargs={}, response_metadata={}, id='2b5076aa-da2c-4105-a8bb-46ea3b442217'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Fdjj80irrLURazobJzXS2KSQ', 'function': {'arguments': '{}', 'name': 'get_stretches'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 492, 'total_tokens': 505, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BPxN98RexKqHtkKiYshOJs3esbzte', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ede59977-58d8-4af8-86cc-8c1f

In [ ]:
messages.append(HumanMessage(content="I'd like to stretch for 20 minutes."))

AttributeError: 'AddableValuesDict' object has no attribute 'append'

In [ ]:
from langgraph.graph.message import add_messages

new_message = HumanMessage(content="I'd like to stretch for 20 minutes.")

add_messages(messages, new_message)

ValueError: Message dict must contain 'role' and 'content' keys, got {'messages': [HumanMessage(content="Hello, I'm feeling discomfort in my hips. I'm hoping you can help me create a stretching routine to alleviate some of this pain and improve my flexibility.", additional_kwargs={}, response_metadata={}, id='2b5076aa-da2c-4105-a8bb-46ea3b442217'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Fdjj80irrLURazobJzXS2KSQ', 'function': {'arguments': '{}', 'name': 'get_stretches'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 492, 'total_tokens': 505, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BPxN98RexKqHtkKiYshOJs3esbzte', 'finish_reason': 't

In [ ]:
type(messages)

langgraph.pregel.io.AddableValuesDict

In [ ]:
messages.values()

dict_values([[HumanMessage(content="Hello, I'm feeling discomfort in my hips. I'm hoping you can help me create a stretching routine to alleviate some of this pain and improve my flexibility.", additional_kwargs={}, response_metadata={}, id='2b5076aa-da2c-4105-a8bb-46ea3b442217'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Fdjj80irrLURazobJzXS2KSQ', 'function': {'arguments': '{}', 'name': 'get_stretches'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 492, 'total_tokens': 505, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BPxN98RexKqHtkKiYshOJs3esbzte', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ede59977-58d8-4af8-86cc-8c1f18